### Data Cleaning

In [1]:
import csv
error_table = "YTFErrors.csv"
error_list = []
with open(error_table) as csvfile:
    csv_reader = csv.reader(csvfile)  
    error_header = next(csv_reader)  
    for row in csv_reader:  
        error_list.append(row[0])

In [2]:
import re
temp = []
for error in error_list:
    temp.append(re.split(r'\W+', error)[0])
error_list = temp

### Name List

In [3]:
import os
from tqdm import tqdm_notebook
name_list = []
for root, dirs, files in tqdm_notebook(os.walk("frame_images_DB")):
    for file in files:
        if file.endswith(".jpg"):
            name = root.split('/')[1]
            if name not in error_list:
                name_list.append(name)


In [4]:
print("Katharine_Hepburn" in error_list)
print("Katharine_Hepburn" in name_list)

# tested OK

True
False


In [5]:
len(set(name_list))

1542

### Train_test_split & Shrinkage

In [6]:
# list first folders of each person as the training data
temp_list = list(set(name_list))
train_set = set()
for root, dirs, files in tqdm_notebook(os.walk("frame_images_DB")):
    for file in files:
        if file.endswith(".jpg"):
            name = root.split('/')[1]            
            if name in temp_list:
                train_set.add(root)
                temp_list.remove(name)

In [7]:
len(train_set)

1542

In [8]:
# list second folders of applicable person as the testing data
temp_list = list(set(name_list))
test_set = set()
for root, dirs, files in tqdm_notebook(os.walk("frame_images_DB")):
    for file in files:
        if file.endswith(".jpg"):
            name = root.split('/')[1]            
            if (name in temp_list) & (root not in train_set):
                test_set.add(root)
                temp_list.remove(name)

In [9]:
len(test_set)

951

In [10]:
# For Train_set Randomly select frames, 48 per person
import random
train_addrs = []
for path in train_set:
    file = random.sample(os.listdir(path),48)
    for i in file:
        train_addrs.append(path+'/'+i)

In [11]:
len(train_addrs)

74016

In [12]:
# For Test_set Randomly select frames, 10 per person
import random
test_addrs = []
for path in test_set:
    file = random.sample(os.listdir(path),10)
    for i in file:
        test_addrs.append(path+'/'+i)

In [13]:
len(test_addrs)

9510

## Face Crop

In [9]:
# generate shape
train_shape = (len(train_addrs), 24, 24, 3)
test_shape = (len(test_addrs), 24, 24, 3)

In [10]:
# generate labels
train_labels = [i.split('/')[1] for i in train_addrs]
test_labels = [i.split('/')[1] for i in test_addrs]

In [45]:
import cv2

#create a CascadeClassifier Object
cascadePath = "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(cascadePath)
#checking to see if classifier has been loaded properly
print(face_cascade)

<CascadeClassifier 0x13b6efed0>


In [18]:
test = "frame_images_DB/Aaron_Eckhart/0/0.555.jpg"

In [44]:
img = cv2.imread(test)
gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv/modules/objdetect/src/cascadedetect.cpp:1698: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


In [27]:
img.shape

(360, 480, 3)

In [37]:
import glob

for file in train_addrs:
    
    image=cv2.imread(file)
    #reading the image as gray scale
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the image
    #search for the face co-ordinates in the image
    #detect multiscale is the method to search for the face rectangle co-ordinates 
    #scale factor =1.1 decreases the shape value by 5% until the face is found. The smaller this value, the more accuracy
    #we may want to hyper parameter tune with scaleFactor and neighbors
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5
#         minSize=(30, 30)
    )
    #print("Found {0} faces!".format(len(faces)))
    
    #Crop padding
    #may not be needed
    left = 10
    right = 10
    top = 10
    bottom = 10
    
    #draw a rectange around the faces
    for (x, y, w, h) in faces:
        print(x, y, w, h)
        #method to create the face rectangle
        #cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 23)
    
    #crop the image
    image  = image[y-top:y+h+bottom, x-left:x+w+right]

error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv/modules/objdetect/src/cascadedetect.cpp:1698: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


### HDF5

In [11]:
# Initialize hdf5
import numpy as np
import h5py
hdf5_path = 'dataset.hdf5'
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_img", train_shape, np.int8)
hdf5_file.create_dataset("test_img", test_shape, np.int8)

<HDF5 dataset "test_img": shape (15312, 24, 24, 3), type "|i1">

In [12]:
hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
dt = h5py.special_dtype(vlen=str) 
hdf5_file.create_dataset("train_labels", (len(train_addrs),), dt)
hdf5_file["train_labels"][...] = train_labels
hdf5_file.create_dataset("test_labels", (len(test_addrs),), dt)
hdf5_file["test_labels"][...] = test_labels

### Load Images

In [13]:
import cv2
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in tqdm_notebook(range(len(train_addrs))):
    # read an image and resize to (24, 24)
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (24, 24), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here

    # save the image and calculate the mean so far
    hdf5_file["train_img"][i, ...] = img[None]
    mean += img / float(len(train_labels))

# loop over test addresses
for i in tqdm_notebook(range(len(test_addrs))):
    # read an image and resize to (24, 24)
    # cv2 load images as BGR, convert it to RGB
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (24, 24), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here

    # save the image
    hdf5_file["test_img"][i, ...] = img[None]
# save the mean and close the hdf5 file
hdf5_file["train_mean"][...] = mean
hdf5_file.close()

KeyboardInterrupt: 

### Read Data

In [14]:
import h5py
import numpy as np
hdf5_path = 'dataset.hdf5'
subtract_mean = False
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")
# subtract the training mean
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
# Total number of samples
data_num = hdf5_file["train_img"].shape[0]

In [15]:
from keras.preprocessing.image import array_to_img
img_test = array_to_img(hdf5_file["train_img"][0])
img_test

Using TensorFlow backend.


In [16]:
data_num

61248

In [18]:
# convert labels to OHE
from keras import utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(hdf5_file["train_labels"])
encoded_train = encoder.transform(hdf5_file["train_labels"])
encoded_test = encoder.transform(hdf5_file["test_labels"])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_train = utils.to_categorical(encoded_train)
dummy_test = utils.to_categorical(encoded_test)


# Build NN Structure
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

batch_size = 1280
num_classes = dummy_train.shape[1]
epochs = 3
input_shape = (24, 24, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation ='relu',
                 input_shape =input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()


history1 = model.fit(hdf5_file["train_img"], dummy_train,
                     batch_size = batch_size,
                     epochs = epochs,
                     verbose = 2,
                     shuffle = 'batch'
                     )
                     


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________